In [1]:
import pandas as pd
from notdiamond.toolkit import CustomRouter
from dotenv import load_dotenv
load_dotenv()
from src.generate import generate_prompt

In [2]:
df = pd.read_csv("data/results.csv")
df['input']  = df['text'].apply(lambda x: generate_prompt(x).format())
print(df.columns)

Index(['provider', 'set', 'text', 'label', 'response', 'predicted', 'score',
       'input'],
      dtype='object')


In [3]:
llm_providers = [
    "openai/gpt-4o-2024-08-06",
    "openai/gpt-4o-mini-2024-07-18",
    "google/gemini-1.5-pro-latest",
    "mistral/mistral-large-2407",
    "togetherai/DeepSeek-R1",
    "perplexity/sonar"
]

pzn_train = {}
pzn_test = {}
for provider in llm_providers:
    provider_results = df[df["provider"] == provider][["set", "input", "response", "score"]]
    
    # Create train/test/extra split
    train = provider_results[provider_results["set"] == "train"].copy().drop(columns=["set"])
    test = provider_results[provider_results["set"] == "test"].copy().drop(columns=["set"])

    pzn_train[provider] = train
    pzn_test[provider] = test

In [4]:
# Initialize the CustomRouter object for training
trainer = CustomRouter(
    language="english",
    maximize=True  # Indicate if higher scores are better (setting to False indicates the opposite)
)

# Train the model using your dataset
preference_id = trainer.fit(
    dataset=pzn_train, # The dataset containing inputs, responses, and scores
    prompt_column="input", # Column name for the input prompts
    response_column="response", # Column name for the model responses
    score_column="score" # Column name for the scores
)

print("Custom router preference ID: ", preference_id)

Custom router preference ID:  935e585a-2a63-411d-a4da-b103ac4e6422
